In [1]:
# pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Auth at first time
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=132657856087-t2gdh3j2s3fcu1ugngr22ji2e7ksfs62.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [32]:
from __future__ import print_function
import json
import requests
import os
import os.path
import shutil
import errno
import sys
import urllib
import pandas as pd
from IPython.display import display

# facepy
import facepy

def gdrive_get_list():
    drive = GoogleDrive(gauth)

    # Auto-iterate through all files that matches this query
    file_list = drive.ListFile({'q': "'root' in parents"}).GetList()
    for file1 in file_list:
      print('title: %s, id: %s' % (file1['title'], file1['id']))

    # Paginate file lists by specifying number of max results
    for file_list in drive.ListFile({'maxResults': 10}):
      print('Received %s files from Files.list()' % len(file_list)) # <= 10
      for file1 in file_list:
        print('title: %s, id: %s' % (file1['title'], file1['id']))

def gdrive_upload_to_folder(fileName):
    # Create GoogleDrive instance with authenticated GoogleAuth instance.
    drive = GoogleDrive(gauth)

    # according to folder id
    f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": '1zlkUR9mitsedvx9trUjgxh54j9vua4go'}]})

    # Make sure to add the path to the file to upload below.
    f.SetContentFile('./downloads/'+fileName)
    f['title'] = fileName
    f.Upload()
    #print('upload completed')

    # Insert the permission.
    permission = f.InsertPermission({
                            'type': 'anyone',
                            'value': 'anyone',
                            'role': 'reader'})

    #print(f['alternateLink'])
    return f['alternateLink']

def download_file(url, down):
    
    fileName = url.split('/')[4].split('?')[0]
    fileName = urllib.parse.unquote(fileName)
    
    if down:
        # write file
        r = requests.get(url, stream=True)
        with open('./downloads/'+fileName, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
                    f.flush()
        doctype = fileName.split('.')[-1]
    else:
        fileName = ''
        
    return fileName

def write_context(fileName, context):
    # write context to txt
    if context:
        with open('./downloads/'+fileName+'.txt', 'w') as f:
                f.write(context)
    

def dl_fb_files(groupId, accessToken):
    
    # Facebook API
    graph = facepy.GraphAPI(accessToken)
    getString = '/v3.0/' + groupId + '/files?fields=download_link,from,message'
    pages = graph.get(getString, page=True, retry=1, limit=100)
    
    # df to save doc files
    df = pd.DataFrame(columns=['資源標題','年級','科目/單元','資源連結','doctype','內文'])
    # df to save other files
    df_packed = pd.DataFrame(columns=['資源標題','年級','科目/單元','資源連結','doctype','內文'])

    counter = 1
    #print(page)
    
    for page in pages:
        for post in page['data']:
            
            # make a tmp dir
            if os.path.isdir("./downloads"):
                shutil.rmtree('./downloads')
            os.mkdir('./downloads')

            # upload user 
            if 'from' in post:
                from_name = post['from']['name']
            else:
                confrom_nametext = ''

            # context
            if 'message' in post:
                context = post['message']
            else:
                context = ''

            # download link
            download_link = post['download_link']

            # download file
            fileName = download_file(download_link, 5)

            # Write context
            #write_context(fileName, context)

            #print('=====\n{}\n{}\n{}\n{}\n\n'.format(fileName, from_name, context, download_link))
            print(counter,'/',' : ',fileName)
            counter += 1

            fileName_lst = fileName.split('.')
            title = fileName_lst[0]
            doctype = fileName_lst[-1]

            if doctype in ['doc', 'docx', 'ppt', 'pptx', 'pdf']:# for doc files
                # upload to gdrive
                gdrive_link = gdrive_upload_to_folder(fileName)

                df = df.append({'資源標題':title,
                    '年級':'',
                    '科目/單元':'',
                    '資源連結':gdrive_link,
                    'doctype':doctype,
                    '內文':context
                    },ignore_index=True)
                df.to_csv('output.csv', index=False)
            else:# for packed files or others
                df_packed = df_packed.append({'資源標題':title,
                    '年級':'',
                    '科目/單元':'',
                    '資源連結':download_link,
                    'doctype':doctype,
                    '內文':context
                    },ignore_index=True)
                df_packed.to_csv('output_others.csv', index=False)
            
    display(df.head())
    display(df_packed.head())
    
    return df, df_packed

#group_id = '1841438095921683'
group_id = '247170855413829' # old wen

usr_token = 'EAAUQuUxvNIUBAEn7Qp5zTNfN2GAPSrpm15m1LFsd0iaT6EhKdZAlZBvN8g0mGDdLUHfyFm15YfIwHpxi4VAkuWvVbU2rFZAKIEamZClYe4qgeCwKqPQlTL7NSJXG0dB7NO8XjygxKw55XyF5CfFqcOHAXb84IEAc5hNVUWNeDAZDZD'
df, df_packed = dl_fb_files(group_id, usr_token)


1 /  :  心_日記.docx
2 /  :  南一六上第二課鷸蚌相爭教學.doc
3 /  :  Character.pdf
4 /  :  數學日記.rar
5 /  :  履歷自傳小書小檔案--接新班_請孩子介紹自己給你吧!.doc
6 /  :  康軒二上(一~四課).rar
7 /  :  二上第01課狗蟻備課.doc
8 /  :  一上第01課來去讀冊備課.doc
9 /  :  六上第01課叫出我的名臺灣備課.doc
10 /  :  溫老師彭老師金門分享筆記.docx
11 /  :  段老師8.21研習筆記.docx
12 /  :  圓面積推演.pptx
13 /  :  1-100的質數.pptx
14 /  :  (康軒版)心智圖一到六只有PDF檔.rar
15 /  :  104(一)135年級預習單word及PDF檔(康軒版).rar
16 /  :  104(一)一到六年級課文word檔(康軒版).rar
17 /  :  預習作業+說明.docx
18 /  :  2上康軒課文預測單.docx
19 /  :  人格特质卡简体更新.pdf
20 /  :  理解篇章的解构分析方式.pdf
21 /  :  康軒六上第一課朱子治家格言選 回饋單.doc
22 /  :  104五上翰林版1-5課資料.rar
23 /  :  溫美玉老師圓夢繪本3.rar
24 /  :  溫美玉老師圓夢繪本2.rar
25 /  :  溫美玉老師圓夢繪本1.rar
26 /  :  段淑如老師研習紀錄 104.docx
27 /  :  104五上翰林第五課棉花上的沉睡者.doc
28 /  :  104五上翰林第四課黑白間的光彩.doc
29 /  :  104五上翰林第三課一池子的綠.doc
30 /  :  104五上翰林第二課湖邊散步.doc
31 /  :  104五上翰林第一課貝殼砂.doc
32 /  :  104五上翰林文本分析.doc
33 /  :  翰林四上說明文.docx
34 /  :  翰林五上1-7課預習單結構單.rar
35 /  :  翰林六上預習單結構單.rar
36 /  :  翰林三上課文分析.rar
37 /  :  20150813六甲上學期預習A本.docx
38 /  :  康軒國語三上預習單1 (

ApiRequestError: <HttpError 500 when requesting https://www.googleapis.com/drive/v2/files/1B6NquEH7rRSBiIrqXBwdDHfNRUylynBY/permissions?alt=json returned "Internal Error">

In [16]:
#https://lookaside.fbsbx.com/file/2-2.ppt?token=AWyebSAPL2lcDrxpSk_YYY2dmaHJPXKq2GDOiR520mkkXsIepoCjA3yvGW0KvfuF22qH1y9hVXrkGt2nMVdnJ8sn4OZgwTb5fyM4q42zmB90kmWvNJfnVB6Yrb3ZEI0u0Ux-InhBYZS7mURf6G3DHoozlpqkweXqKfavHonqCHfnok5YA6wYnLwKY0mFOmKzVcw
#https://lookaside.fbsbx.com/file/1%E6%95%B8%E5%AD%B8%E8%A7%A3%E9%A1%8C%E9%96%B1%E8%AE%80%E5%AD%B8%E7%BF%92%E6%B4%BB%E5%8B%95%E8%A8%AD%E8%A8%88_%E5%8D%97%E9%96%80%20%20%E6%9B%BE%E6%98%8E%E5%BE%B7_%E5%9C%8B%E6%96%87%E7%9A%84%E6%A3%8B%E7%9B%A4.docx?token=AWygxtrzMs5jC1AobwBoSqtgAG1pmiKjwS6IT3yU5OOLq9QghHAi99fGUqKBoW6qge6-zDfIQwiytszMsoHGPWBso0dceDRjFW7JsnCRZ0NLbjTXszgc43yXgZ9kAsksdW_Qi6k9-UnNtZBxSwViFn-MWNRq6iTHoBZMv5yMdeX-77jDRT9j47LZWrn6Gfo9N6E

NameError: name 'df' is not defined

In [53]:
df2 = pd.read_csv('output.csv')
df2.head()

,資源標題,年級,科目/單元,資源連結,doctype,內文
0,2-2,NaN,NaN,https://drive.google.com/a/junyiacademy.org/fi...,ppt,這是內文這是內文這是內文這是內文這是內文這是內文這是內文\n123123123\nddddd
1,1數學解題閱讀學習活動設計_南門 曾明德_國文的棋盤,NaN,NaN,https://drive.google.com/a/junyiacademy.org/fi...,docx,NaN
